In [1]:
from dataclasses import dataclass

import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
fhandler = logging.FileHandler(filename='simulate.log', mode='w')
formatter = logging.Formatter('%(message)s')
fhandler.setFormatter(formatter)
logger.addHandler(fhandler)

@dataclass
class Entity:
    id: int
    arrival_time: float
    interarrival_time: float
    service_time: float
    def __repr__(self):
        return f"Entity(\
id={self.id}, \
arrival_time={self.arrival_time}, \
interarrival_time={self.interarrival_time}, \
service_time={self.service_time})"

class FIFO_queue:
    def __init__(self, q_type, entity_list=None):
        self.q_type = q_type
        if entity_list:
            self.queue = entity_list
        else:
            self.queue = []
    def add(self, entity, clock):
        logger.info(f"[{clock}][{self.q_type} queue]: ADD {entity}")
        self.queue.insert(0, entity)
    def pop(self, clock):
        pop_entity = self.queue.pop()
        logger.info(f"[{clock}][{self.q_type} queue]: POP {pop_entity}")
        return pop_entity
    def __repr__(self):
        return f"FIFO_queue({self.queue})"

class Server:
    def __init__(self, shotdown_policy = False):
        self.shotdown_policy = shotdown_policy
        self.shotdown = False
        # self.busy_until = 0
        self.entity = None
    
    def check_shotdown(self, clock = None):
        if self.shotdown_policy == True:
            if clock != None:
                location_in_period = clock % (400 + 400 + 2000)
                if 400 <= location_in_period and location_in_period < (400 + 400):
                    if self.shotdown == False:
                        logger.info(f"[{clock}][Server]: SHOTDOWN, {self.entity} in server")
                    self.shotdown = True
                else:
                    if self.shotdown == True:
                        logger.info(f"[{clock}][Server]: BACK, {self.entity} in server")
                    self.shotdown = False   
    def __repr__(self):
        return f"Server({self.entity})"

In [2]:
@dataclass
class Status:
    clock: int

    server: Server
    arriving_queue: FIFO_queue
    waiting_queue: FIFO_queue
    finished_queue: FIFO_queue

    B_t: list
    Q_t: list
    num_completed_waiting: int
    total_waiting_time_in_queue: list
    
    # area_under_Q_t: int
    # area_under_B_t: int

    unit_in_second: float
    second_pre_unit: int


    def check_Q_t(self):
        self.Q_t.append((self.clock, len(self.waiting_queue.queue)))
        # self.area_under_Q_t += len(self.waiting_queue.queue) * int(self.unit_in_second * self.second_pre_unit)

    def check_arrival(self):
        if self.arriving_queue.queue:
            len_arriving_queue = len(self.arriving_queue.queue)
            # logger.debug(f"check_arrival: {self.clock}, len of arriving_queue: {len_arriving_queue}")
            if self.clock == self.arriving_queue.queue[-1].arrival_time:
                # logger.info(f"[new_arrival]: clock {self.clock}")
                entity = self.arriving_queue.pop(self.clock)
                self.waiting_queue.add(entity, self.clock)

    
    def check_service(self):
        # logger.debug(f"check_service: {self.clock}, server: {self.server}")
        self.server.check_shotdown(self.clock)

        if self.server.entity != None:
            if self.server.shotdown == False:
            # if True:
                self.server.entity.service_time -= int(self.unit_in_second * self.second_pre_unit)
            
            if self.server.entity.service_time == 0: #! Check this
                self.finished_queue.add(self.server.entity, self.clock)
                logger.info(f"[{self.clock}][server]: FINISH {self.server.entity}")
                self.server.entity = None
                # logger.info(f"server: SERVE {self.server.entity}")
        if self.server.entity == None:
            if len(self.waiting_queue.queue) > 0:
                entity = self.waiting_queue.pop(self.clock)
                self.num_completed_waiting += 1
                self.server.entity = entity
                #! self.server.entity.service_time -= int(self.unit_in_second * self.second_pre_unit)
                #! self.total_waiting_time_in_queue += self.clock - entity.arrival_time
                if len(self.total_waiting_time_in_queue) > 0:
                    self.total_waiting_time_in_queue.append((self.clock, self.clock - entity.arrival_time + self.total_waiting_time_in_queue[-1][1])) #! Check this
                else:
                    self.total_waiting_time_in_queue.append((self.clock, self.clock - entity.arrival_time))
                logger.info(f"[{self.clock}][server]: START {self.server.entity}")
        
        server_status = 1 if self.server.entity else 0
        self.B_t.append((self.clock, server_status))
    
    def process(self):
        self.check_arrival()
        self.check_service()
        self.check_Q_t()
        self.clock += int(self.unit_in_second * self.second_pre_unit)
    
    def average_waiting_time_in_queue(self):
        return (self.total_waiting_time_in_queue[-1][1] / self.num_completed_waiting) * self.unit_in_second

    def time_average_number_in_queue(self):
        area_under_Q_t = sum([i[1] for i in self.Q_t[:-1]]) 
        # print(f"area under Q_t: {area_under_Q_t}")
        return area_under_Q_t / (len(self.Q_t[:-1])) #! Check this

    def utilization(self):
        area_under_B_t = sum([i[1] for i in self.B_t[:-1]])
        # print(f"area under B_t: {area_under_B_t}")
        return area_under_B_t / (len(self.B_t[:-1])) #1 Check this
        
    def __repr__(self):
        return f"===========\n\
clock: {self.clock}\n\n\
server: {self.server}\n\n\
arriving_queue: {self.arriving_queue}\n\n\
waiting_queue: {self.waiting_queue}\n\n\
finished_queue: {self.finished_queue}\n\n\
num_completed_waiting: {self.num_completed_waiting}\n\n\
total_waiting_time_in_queue: {self.total_waiting_time_in_queue}\n\
===========\n"
# B_t: {self.B_t}, Q_t: {self.Q_t}, 
    # def __repr__(self):
    #     return f"Status(\
# area_under_Q_t: {self.area_under_Q_t}, area_under_B_t: {self.area_under_B_t}\n\n\

In [8]:
import numpy as np

lambda_ = 0.2682926829
shape = 41

def generate_poisson_arrival_time(lambda_, shape, seed):
    np.random.seed(seed)
    random_poission = np.random.poisson(lambda_, shape)
    # print(random_poission)
    result = []
    for i, count in enumerate(random_poission):
        interval_arrival_time = [i * 100 + rand for rand in np.random.randint(low=0, high=10, size=count)]
        result.extend(interval_arrival_time)
    return sorted(result)


In [9]:
import numpy as np
mean = np.mean([290, 176, 339, 452, 446, 436, 207, 336, 237, 538])
std = np.std([290, 176, 339, 452, 446, 436, 207, 336, 237, 538])

def generate_normal_service_time(mean, std, shape, seed):
    np.random.seed(seed)
    return np.random.normal(mean, std, shape).round().astype(int)


In [10]:
def generate_interarrival_time(arrival_time):
    result = []
    for i in range(1, len(arrival_time)):
        result.append(arrival_time[i] - arrival_time[i - 1])
    result.append(0)
    return result

In [11]:
sample_size = 30
arrival_time_list_list = [generate_poisson_arrival_time(lambda_, shape, i) for i in range(sample_size)]
service_time_list_list = [generate_normal_service_time(mean, std, len(arrival_time), i) for i, arrival_time in enumerate(arrival_time_list_list)]
interarrival_time_list_list = [generate_interarrival_time(arrival_time) for arrival_time in arrival_time_list_list]

In [12]:
with open("arrival_time_list_list.txt", "w") as f:
    f.write(str(arrival_time_list_list))
with open("service_time_list_list.txt", "w") as f:
    f.write(str(service_time_list_list))
with open("interarrival_time_list_list.txt", "w") as f:
    f.write(str(interarrival_time_list_list))

In [13]:
import pandas as pd
df_base = pd.DataFrame(columns=["arrival_time_list", "interarrival_time_list", "service_time_list", "average_waiting_time_in_queue", "time_average_number_in_queue", "utilization"])

In [14]:

for arrival_time_list, interarrival_time_list, service_time_list in zip(arrival_time_list_list.copy(), interarrival_time_list_list.copy(), service_time_list_list.copy()):


    entity_list = [Entity(i + 1, arrival_time, interarrival_time, service_time) for i, (arrival_time, interarrival_time, service_time) in enumerate(zip(arrival_time_list, interarrival_time_list, service_time_list))]
    entity_list.reverse()

    system_status = Status(
        clock = int(0.00*100),
        server = Server(),
        arriving_queue = FIFO_queue("arring", entity_list.copy()),
        waiting_queue = FIFO_queue("waiting", None),
        finished_queue = FIFO_queue("finished", None),
        num_completed_waiting = 0,
        B_t = [], # Server status (time, status)
        Q_t = [], # Queue status (time, num_in_queue)
        total_waiting_time_in_queue = [],
        unit_in_second = 0.01,
        second_pre_unit = 100,
    )
    # print(system_status)
    while system_status.clock <= int(20.00*100):
        system_status.process()

    # print(system_status)

    print(f"average waiting time in queue: {system_status.average_waiting_time_in_queue()}")
    print(f"time average number in queue: {system_status.time_average_number_in_queue()}")
    print(f"utilization: {system_status.utilization()}")

    logger.info(f"average waiting time in queue: {system_status.average_waiting_time_in_queue()}")
    logger.info(f"time average number in queue: {system_status.time_average_number_in_queue()}")
    logger.info(f"utilization: {system_status.utilization()}")
    df_base = pd.concat([df_base, pd.DataFrame([[arrival_time_list, interarrival_time_list, service_time_list, system_status.average_waiting_time_in_queue(), system_status.time_average_number_in_queue(), system_status.utilization()]], columns=df_base.columns)], ignore_index=True)

# while system_status.clock <= int(1.73*100):
#    system_status.process()
#    if system_status.clock == int(1.73*100):
#       print(system_status)
#    system_status.clock += int(system_status.unit_in_second * system_status.second_pre_unit)


INFO:__main__:[704][arring queue]: POP Entity(id=1, arrival_time=704, interarrival_time=2, service_time=547)
INFO:__main__:[704][waiting queue]: ADD Entity(id=1, arrival_time=704, interarrival_time=2, service_time=547)
INFO:__main__:[704][waiting queue]: POP Entity(id=1, arrival_time=704, interarrival_time=2, service_time=547)
INFO:__main__:[704][server]: START Entity(id=1, arrival_time=704, interarrival_time=2, service_time=547)
INFO:__main__:[706][arring queue]: POP Entity(id=2, arrival_time=706, interarrival_time=98, service_time=391)
INFO:__main__:[706][waiting queue]: ADD Entity(id=2, arrival_time=706, interarrival_time=98, service_time=391)
INFO:__main__:[804][arring queue]: POP Entity(id=3, arrival_time=804, interarrival_time=200, service_time=457)
INFO:__main__:[804][waiting queue]: ADD Entity(id=3, arrival_time=804, interarrival_time=200, service_time=457)
INFO:__main__:[1004][arring queue]: POP Entity(id=4, arrival_time=1004, interarrival_time=299, service_time=601)
INFO:__ma

average waiting time in queue: 4.61
time average number in queue: 1.836
utilization: 0.648
average waiting time in queue: 0.0
time average number in queue: 0.049
utilization: 0.3155
average waiting time in queue: 0.52
time average number in queue: 0.052
utilization: 0.246
average waiting time in queue: 2.71
time average number in queue: 0.271
utilization: 0.472
average waiting time in queue: 4.274285714285715
time average number in queue: 2.0415
utilization: 0.997
average waiting time in queue: 2.58
time average number in queue: 0.516
utilization: 0.8215
average waiting time in queue: 0.7266666666666667
time average number in queue: 0.109
utilization: 0.5545
average waiting time in queue: 1.7333333333333334
time average number in queue: 0.6185
utilization: 0.949
average waiting time in queue: 0.8675
time average number in queue: 0.1735
utilization: 0.569


INFO:__main__:[1102][arring queue]: POP Entity(id=1, arrival_time=1102, interarrival_time=7, service_time=346)
INFO:__main__:[1102][waiting queue]: ADD Entity(id=1, arrival_time=1102, interarrival_time=7, service_time=346)
INFO:__main__:[1102][waiting queue]: POP Entity(id=1, arrival_time=1102, interarrival_time=7, service_time=346)
INFO:__main__:[1102][server]: START Entity(id=1, arrival_time=1102, interarrival_time=7, service_time=346)
INFO:__main__:[1109][arring queue]: POP Entity(id=2, arrival_time=1109, interarrival_time=292, service_time=313)
INFO:__main__:[1109][waiting queue]: ADD Entity(id=2, arrival_time=1109, interarrival_time=292, service_time=313)
INFO:__main__:[1401][arring queue]: POP Entity(id=3, arrival_time=1401, interarrival_time=502, service_time=219)
INFO:__main__:[1401][waiting queue]: ADD Entity(id=3, arrival_time=1401, interarrival_time=502, service_time=219)
INFO:__main__:[1448][finished queue]: ADD Entity(id=1, arrival_time=1102, interarrival_time=7, service_t

average waiting time in queue: 1.94
time average number in queue: 0.388
utilization: 0.449
average waiting time in queue: 0.8200000000000001
time average number in queue: 0.164
utilization: 0.7195
average waiting time in queue: 4.876
time average number in queue: 1.219
utilization: 0.599
average waiting time in queue: 4.768
time average number in queue: 1.539
utilization: 0.749
average waiting time in queue: 6.6033333333333335
time average number in queue: 2.5245
utilization: 0.998


INFO:__main__:[103][waiting queue]: ADD Entity(id=1, arrival_time=103, interarrival_time=298, service_time=522)
INFO:__main__:[103][waiting queue]: POP Entity(id=1, arrival_time=103, interarrival_time=298, service_time=522)
INFO:__main__:[103][server]: START Entity(id=1, arrival_time=103, interarrival_time=298, service_time=522)
INFO:__main__:[401][arring queue]: POP Entity(id=2, arrival_time=401, interarrival_time=404, service_time=355)
INFO:__main__:[401][waiting queue]: ADD Entity(id=2, arrival_time=401, interarrival_time=404, service_time=355)
INFO:__main__:[625][finished queue]: ADD Entity(id=1, arrival_time=103, interarrival_time=298, service_time=0)
INFO:__main__:[625][server]: FINISH Entity(id=1, arrival_time=103, interarrival_time=298, service_time=0)
INFO:__main__:[625][waiting queue]: POP Entity(id=2, arrival_time=401, interarrival_time=404, service_time=355)
INFO:__main__:[625][server]: START Entity(id=2, arrival_time=401, interarrival_time=404, service_time=355)
INFO:__mai

average waiting time in queue: 1.038
time average number in queue: 0.2595
utilization: 0.849
average waiting time in queue: 0.4475
time average number in queue: 0.0895
utilization: 0.6555
average waiting time in queue: 0.0
time average number in queue: 0.0
utilization: 0.18
average waiting time in queue: 0.1
time average number in queue: 0.015
utilization: 0.4645
average waiting time in queue: 4.264
time average number in queue: 1.2605
utilization: 0.898
average waiting time in queue: 1.2133333333333334
time average number in queue: 0.182
utilization: 0.3885


INFO:__main__:[1104][waiting queue]: ADD Entity(id=4, arrival_time=1104, interarrival_time=502, service_time=79)
INFO:__main__:[1390][finished queue]: ADD Entity(id=3, arrival_time=1004, interarrival_time=100, service_time=0)
INFO:__main__:[1390][server]: FINISH Entity(id=3, arrival_time=1004, interarrival_time=100, service_time=0)
INFO:__main__:[1390][waiting queue]: POP Entity(id=4, arrival_time=1104, interarrival_time=502, service_time=79)
INFO:__main__:[1390][server]: START Entity(id=4, arrival_time=1104, interarrival_time=502, service_time=79)
INFO:__main__:[1469][finished queue]: ADD Entity(id=4, arrival_time=1104, interarrival_time=502, service_time=0)
INFO:__main__:[1469][server]: FINISH Entity(id=4, arrival_time=1104, interarrival_time=502, service_time=0)
INFO:__main__:[1606][arring queue]: POP Entity(id=5, arrival_time=1606, interarrival_time=2, service_time=222)
INFO:__main__:[1606][waiting queue]: ADD Entity(id=5, arrival_time=1606, interarrival_time=2, service_time=222)
I

average waiting time in queue: 1.5766666666666667
time average number in queue: 0.573
utilization: 0.8365
average waiting time in queue: 1.0666666666666667
time average number in queue: 0.16
utilization: 0.4465
average waiting time in queue: 0.7120000000000001
time average number in queue: 0.178
utilization: 0.7555
average waiting time in queue: 1.875
time average number in queue: 1.1175
utilization: 0.7105
average waiting time in queue: 1.3666666666666667
time average number in queue: 0.458
utilization: 0.81
average waiting time in queue: 0.0
time average number in queue: 0.0
utilization: 0.2845
average waiting time in queue: 1.705
time average number in queue: 0.4835
utilization: 0.5465
average waiting time in queue: 6.222
time average number in queue: 2.8465
utilization: 0.949


INFO:__main__:[1505][arring queue]: POP Entity(id=8, arrival_time=1505, interarrival_time=97, service_time=311)
INFO:__main__:[1505][waiting queue]: ADD Entity(id=8, arrival_time=1505, interarrival_time=97, service_time=311)
INFO:__main__:[1602][arring queue]: POP Entity(id=9, arrival_time=1602, interarrival_time=7, service_time=250)
INFO:__main__:[1602][waiting queue]: ADD Entity(id=9, arrival_time=1602, interarrival_time=7, service_time=250)
INFO:__main__:[1609][arring queue]: POP Entity(id=10, arrival_time=1609, interarrival_time=199, service_time=287)
INFO:__main__:[1609][waiting queue]: ADD Entity(id=10, arrival_time=1609, interarrival_time=199, service_time=287)
INFO:__main__:[1658][finished queue]: ADD Entity(id=3, arrival_time=604, interarrival_time=4, service_time=0)
INFO:__main__:[1658][server]: FINISH Entity(id=3, arrival_time=604, interarrival_time=4, service_time=0)
INFO:__main__:[1658][waiting queue]: POP Entity(id=4, arrival_time=608, interarrival_time=95, service_time=2

average waiting time in queue: 6.3660000000000005
time average number in queue: 3.422
utilization: 0.7955
average waiting time in queue: 0.0
time average number in queue: 0.1435
utilization: 0.559


In [15]:
logger.info(f"============================================================================")
logger.info(f"============================================================================")
logger.info(f"============================================================================")
logger.info(f"============================================================================")
logger.info(f"============================================================================")
logger.info(f"============================================================================")
logger.info(f"============================================================================")

INFO:__main__:============================================================================
INFO:__main__:============================================================================
INFO:__main__:============================================================================
INFO:__main__:============================================================================
INFO:__main__:============================================================================
INFO:__main__:============================================================================
INFO:__main__:============================================================================


In [16]:
df_stop = pd.DataFrame(columns=["arrival_time_list", "interarrival_time_list", "service_time_list", "average_waiting_time_in_queue", "time_average_number_in_queue", "utilization"])

In [17]:
for arrival_time_list, interarrival_time_list, service_time_list in zip(arrival_time_list_list.copy(), interarrival_time_list_list.copy(), service_time_list_list.copy()):

    entity_list = [Entity(i + 1, arrival_time, interarrival_time, service_time) for i, (arrival_time, interarrival_time, service_time) in enumerate(zip(arrival_time_list, interarrival_time_list, service_time_list))]
    entity_list.reverse()

    system_status2 = Status(
        clock = int(0.00*100),
        server = Server(shotdown_policy=True),
        arriving_queue = FIFO_queue("arring", entity_list.copy()),
        waiting_queue = FIFO_queue("waiting", None),
        finished_queue = FIFO_queue("finished", None),
        num_completed_waiting = 0,
        B_t = [], # Server status (time, status)
        Q_t = [], # Queue status (time, num_in_queue)
        total_waiting_time_in_queue = [],
        unit_in_second = 0.01,
        second_pre_unit = 100,
    )
    # print(system_status2)
    while system_status2.clock <= int(20.00*100):
        system_status2.process()
        # if system_status.server.shotdown == True:
        #   print(system_status)
        #   break

    # print(system_status2)

    print(f"average waiting time in queue: {system_status2.average_waiting_time_in_queue()}")
    print(f"time average number in queue: {system_status2.time_average_number_in_queue()}")
    print(f"utilization: {system_status2.utilization()}")

    logger.info(f"average waiting time in queue: {system_status2.average_waiting_time_in_queue()}")
    logger.info(f"time average number in queue: {system_status2.time_average_number_in_queue()}")
    logger.info(f"utilization: {system_status2.utilization()}")
    df_stop = pd.concat([df_stop, pd.DataFrame([[arrival_time_list, interarrival_time_list, service_time_list, system_status2.average_waiting_time_in_queue(), system_status2.time_average_number_in_queue(), system_status2.utilization()]], columns=df_stop.columns)], ignore_index=True)

INFO:__main__:[400][Server]: SHOTDOWN, None in server
INFO:__main__:[704][arring queue]: POP Entity(id=1, arrival_time=704, interarrival_time=2, service_time=547)
INFO:__main__:[704][waiting queue]: ADD Entity(id=1, arrival_time=704, interarrival_time=2, service_time=547)
INFO:__main__:[704][waiting queue]: POP Entity(id=1, arrival_time=704, interarrival_time=2, service_time=547)
INFO:__main__:[704][server]: START Entity(id=1, arrival_time=704, interarrival_time=2, service_time=547)
INFO:__main__:[706][arring queue]: POP Entity(id=2, arrival_time=706, interarrival_time=98, service_time=391)
INFO:__main__:[706][waiting queue]: ADD Entity(id=2, arrival_time=706, interarrival_time=98, service_time=391)
INFO:__main__:[800][Server]: BACK, Entity(id=1, arrival_time=704, interarrival_time=2, service_time=547) in server
INFO:__main__:[804][arring queue]: POP Entity(id=3, arrival_time=804, interarrival_time=200, service_time=457)
INFO:__main__:[804][waiting queue]: ADD Entity(id=3, arrival_time

average waiting time in queue: 5.243333333333334
time average number in queue: 1.931
utilization: 0.648
average waiting time in queue: 0.0
time average number in queue: 0.049
utilization: 0.3155
average waiting time in queue: 0.52
time average number in queue: 0.052
utilization: 0.246
average waiting time in queue: 4.705
time average number in queue: 0.4705
utilization: 0.6715
average waiting time in queue: 5.417999999999999
time average number in queue: 2.844
utilization: 0.997
average waiting time in queue: 5.58
time average number in queue: 1.116
utilization: 0.9475
average waiting time in queue: 0.7266666666666667
time average number in queue: 0.109
utilization: 0.7545
average waiting time in queue: 4.12
time average number in queue: 1.4165
utilization: 0.949


INFO:__main__:[9][arring queue]: POP Entity(id=2, arrival_time=9, interarrival_time=1491, service_time=470)
INFO:__main__:[9][waiting queue]: ADD Entity(id=2, arrival_time=9, interarrival_time=1491, service_time=470)
INFO:__main__:[356][finished queue]: ADD Entity(id=1, arrival_time=0, interarrival_time=9, service_time=0)
INFO:__main__:[356][server]: FINISH Entity(id=1, arrival_time=0, interarrival_time=9, service_time=0)
INFO:__main__:[356][waiting queue]: POP Entity(id=2, arrival_time=9, interarrival_time=1491, service_time=470)
INFO:__main__:[356][server]: START Entity(id=2, arrival_time=9, interarrival_time=1491, service_time=470)
INFO:__main__:[400][Server]: SHOTDOWN, Entity(id=2, arrival_time=9, interarrival_time=1491, service_time=427) in server
INFO:__main__:[800][Server]: BACK, Entity(id=2, arrival_time=9, interarrival_time=1491, service_time=427) in server
INFO:__main__:[1226][finished queue]: ADD Entity(id=2, arrival_time=9, interarrival_time=1491, service_time=0)
INFO:__mai

average waiting time in queue: 0.8675
time average number in queue: 0.1735
utilization: 0.769
average waiting time in queue: 1.94
time average number in queue: 0.388
utilization: 0.449
average waiting time in queue: 0.8200000000000001
time average number in queue: 0.164
utilization: 0.9195
average waiting time in queue: 4.876
time average number in queue: 1.219
utilization: 0.599
average waiting time in queue: 7.144
time average number in queue: 2.133
utilization: 0.749
average waiting time in queue: 8.06
time average number in queue: 3.1555
utilization: 0.998
average waiting time in queue: 4.0925
time average number in queue: 1.018
utilization: 0.9485


INFO:__main__:utilization: 0.9485
INFO:__main__:[7][arring queue]: POP Entity(id=1, arrival_time=7, interarrival_time=895, service_time=310)
INFO:__main__:[7][waiting queue]: ADD Entity(id=1, arrival_time=7, interarrival_time=895, service_time=310)
INFO:__main__:[7][waiting queue]: POP Entity(id=1, arrival_time=7, interarrival_time=895, service_time=310)
INFO:__main__:[7][server]: START Entity(id=1, arrival_time=7, interarrival_time=895, service_time=310)
INFO:__main__:[317][finished queue]: ADD Entity(id=1, arrival_time=7, interarrival_time=895, service_time=0)
INFO:__main__:[317][server]: FINISH Entity(id=1, arrival_time=7, interarrival_time=895, service_time=0)
INFO:__main__:[400][Server]: SHOTDOWN, None in server
INFO:__main__:[800][Server]: BACK, None in server
INFO:__main__:[902][arring queue]: POP Entity(id=2, arrival_time=902, interarrival_time=205, service_time=384)
INFO:__main__:[902][waiting queue]: ADD Entity(id=2, arrival_time=902, interarrival_time=205, service_time=384)


average waiting time in queue: 0.4475
time average number in queue: 0.0895
utilization: 0.6555
average waiting time in queue: 0.0
time average number in queue: 0.0
utilization: 0.18
average waiting time in queue: 1.9333333333333336
time average number in queue: 0.29
utilization: 0.6625
average waiting time in queue: 6.9725
time average number in queue: 1.9365
utilization: 0.898
average waiting time in queue: 1.5433333333333334
time average number in queue: 0.2315
utilization: 0.438
average waiting time in queue: 3.85
time average number in queue: 1.2585
utilization: 0.949
average waiting time in queue: 1.0666666666666667
time average number in queue: 0.16
utilization: 0.4465
average waiting time in queue: 2.11
time average number in queue: 0.6195
utilization: 0.846
average waiting time in queue: 1.875
time average number in queue: 1.1175
utilization: 0.9105


INFO:__main__:[5][server]: START Entity(id=1, arrival_time=5, interarrival_time=98, service_time=497)
INFO:__main__:[103][arring queue]: POP Entity(id=2, arrival_time=103, interarrival_time=301, service_time=258)
INFO:__main__:[103][waiting queue]: ADD Entity(id=2, arrival_time=103, interarrival_time=301, service_time=258)
INFO:__main__:[400][Server]: SHOTDOWN, Entity(id=1, arrival_time=5, interarrival_time=98, service_time=103) in server
INFO:__main__:[404][arring queue]: POP Entity(id=3, arrival_time=404, interarrival_time=601, service_time=310)
INFO:__main__:[404][waiting queue]: ADD Entity(id=3, arrival_time=404, interarrival_time=601, service_time=310)
INFO:__main__:[800][Server]: BACK, Entity(id=1, arrival_time=5, interarrival_time=98, service_time=103) in server
INFO:__main__:[902][finished queue]: ADD Entity(id=1, arrival_time=5, interarrival_time=98, service_time=0)
INFO:__main__:[902][server]: FINISH Entity(id=1, arrival_time=5, interarrival_time=98, service_time=0)
INFO:__ma

average waiting time in queue: 3.736666666666667
time average number in queue: 1.169
utilization: 0.9975
average waiting time in queue: 0.0
time average number in queue: 0.0
utilization: 0.2845
average waiting time in queue: 2.705
time average number in queue: 0.6835
utilization: 0.7465
average waiting time in queue: 7.9975000000000005
time average number in queue: 3.5905
utilization: 0.949
average waiting time in queue: 5.74
time average number in queue: 4.036
utilization: 0.7955
average waiting time in queue: 0.0
time average number in queue: 0.1435
utilization: 0.6555


In [18]:
df_base.to_csv("base_repetition.csv")

In [19]:
df_stop.to_csv("stop_repetition.csv")

In [20]:
df_base

,arrival_time_list,interarrival_time_list,service_time_list,average_waiting_time_in_queue,time_average_number_in_queue,utilization
0,"[704, 706, 804, 1004, 1303, 1404, 1404, 1608, ...","[2, 98, 200, 299, 101, 0, 204, 296, 299, 604, 0]","[547, 391, 457, 601, 558, 234, 454, 328, 334, ...",4.610000,1.8360,0.6480
1,"[1308, 1900, 1902, 2107, 2107, 2409, 2607, 300...","[592, 2, 205, 0, 302, 198, 396, 197, 208, 199, 0]","[531, 276, 286, 224, 444, 84, 544, 259, 382, 3...",0.000000,0.0490,0.3155
2,"[1508, 1702, 3409, 3706, 3805]","[194, 1707, 297, 99, 0]","[298, 339, 103, 532, 142]",0.520000,0.0520,0.2460
3,"[400, 407, 2305, 2600, 2601, 2704, 3901]","[7, 1898, 295, 1, 103, 1197, 0]","[549, 395, 357, 134, 314, 305, 336]",2.710000,0.2710,0.4720
4,"[6, 107, 400, 704, 806, 808, 1304, 1408, 1501,...","[101, 293, 304, 102, 2, 496, 104, 93, 708, 996...","[351, 403, 232, 425, 298, 165, 272, 414, 384, ...",4.274286,2.0415,0.9970
5,"[105, 202, 400, 1107, 2306, 2307, 2700, 3200, ...","[97, 198, 707, 1199, 1, 393, 500, 108, 97, 200...","[396, 308, 622, 317, 358, 526, 242, 278, 367, ...",2.580000,0.5160,0.8215
6,"[9, 101, 1302, 1302, 1409, 1705, 2104, 2401, 2...","[92, 1201, 0, 107, 296, 399, 297, 405, 597, 49...","[310, 429, 370, 243, 63, 450, 474, 173, 532, 2...",0.726667,0.1090,0.5545
7,"[102, 303, 902, 1202, 1405, 1607, 1803, 2800, ...","[201, 599, 300, 203, 202, 196, 997, 1109, 0]","[538, 293, 349, 392, 256, 346, 346, 146, 462]",1.733333,0.6185,0.9490
8,"[0, 9, 1500, 1800, 2306, 2409, 2807, 3104, 340...","[9, 1491, 300, 506, 103, 398, 297, 303, 197, 5...","[356, 470, 124, 188, 84, 620, 542, 597, 436, 4...",0.867500,0.1735,0.5690
9,"[1102, 1109, 1401, 1903, 2000, 2101, 2300, 240...","[7, 292, 502, 97, 101, 199, 100, 2, 202, 4, 69...","[346, 313, 219, 344, 303, 291, 173, 290, 318, ...",1.940000,0.3880,0.4490


In [21]:
df_stop

,arrival_time_list,interarrival_time_list,service_time_list,average_waiting_time_in_queue,time_average_number_in_queue,utilization
0,"[704, 706, 804, 1004, 1303, 1404, 1404, 1608, ...","[2, 98, 200, 299, 101, 0, 204, 296, 299, 604, 0]","[547, 391, 457, 601, 558, 234, 454, 328, 334, ...",5.243333,1.9310,0.6480
1,"[1308, 1900, 1902, 2107, 2107, 2409, 2607, 300...","[592, 2, 205, 0, 302, 198, 396, 197, 208, 199, 0]","[531, 276, 286, 224, 444, 84, 544, 259, 382, 3...",0.000000,0.0490,0.3155
2,"[1508, 1702, 3409, 3706, 3805]","[194, 1707, 297, 99, 0]","[298, 339, 103, 532, 142]",0.520000,0.0520,0.2460
3,"[400, 407, 2305, 2600, 2601, 2704, 3901]","[7, 1898, 295, 1, 103, 1197, 0]","[549, 395, 357, 134, 314, 305, 336]",4.705000,0.4705,0.6715
4,"[6, 107, 400, 704, 806, 808, 1304, 1408, 1501,...","[101, 293, 304, 102, 2, 496, 104, 93, 708, 996...","[351, 403, 232, 425, 298, 165, 272, 414, 384, ...",5.418000,2.8440,0.9970
5,"[105, 202, 400, 1107, 2306, 2307, 2700, 3200, ...","[97, 198, 707, 1199, 1, 393, 500, 108, 97, 200...","[396, 308, 622, 317, 358, 526, 242, 278, 367, ...",5.580000,1.1160,0.9475
6,"[9, 101, 1302, 1302, 1409, 1705, 2104, 2401, 2...","[92, 1201, 0, 107, 296, 399, 297, 405, 597, 49...","[310, 429, 370, 243, 63, 450, 474, 173, 532, 2...",0.726667,0.1090,0.7545
7,"[102, 303, 902, 1202, 1405, 1607, 1803, 2800, ...","[201, 599, 300, 203, 202, 196, 997, 1109, 0]","[538, 293, 349, 392, 256, 346, 346, 146, 462]",4.120000,1.4165,0.9490
8,"[0, 9, 1500, 1800, 2306, 2409, 2807, 3104, 340...","[9, 1491, 300, 506, 103, 398, 297, 303, 197, 5...","[356, 470, 124, 188, 84, 620, 542, 597, 436, 4...",0.867500,0.1735,0.7690
9,"[1102, 1109, 1401, 1903, 2000, 2101, 2300, 240...","[7, 292, 502, 97, 101, 199, 100, 2, 202, 4, 69...","[346, 313, 219, 344, 303, 291, 173, 290, 318, ...",1.940000,0.3880,0.4490


In [22]:
df_base.describe()

,average_waiting_time_in_queue,time_average_number_in_queue,utilization
count,30.000000,30.000000,30.000000
mean,2.166065,0.756317,0.640533
std,2.071760,0.931467,0.230615
min,0.000000,0.000000,0.180000
25%,0.715667,0.147625,0.466375
50%,1.471667,0.329500,0.651750
75%,3.875500,1.193625,0.818625
max,6.603333,3.422000,0.998000


In [23]:
df_stop.describe()

,average_waiting_time_in_queue,time_average_number_in_queue,utilization
count,30.000000,30.000000,30.000000
mean,3.136350,1.052150,0.712517
std,2.570097,1.137246,0.246090
min,0.000000,0.000000,0.180000
25%,0.831875,0.161000,0.611250
50%,2.407500,0.651500,0.751750
75%,5.151500,1.377000,0.940500
max,8.060000,4.036000,0.998000
